In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
from scipy.stats import f

In [4]:
tillamook = [0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735,
             0.0659, 0.0923, 0.0836]

newport = [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835,
           0.0725]


petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]

In [5]:
tot_mean=(len(tillamook)*np.mean(tillamook)+len(newport)*np.mean(newport)+len(petersburg)*np.mean(petersburg))/(len(tillamook)+len(newport)+len(petersburg))



In [32]:
SSW=(len(tillamook)-1)*np.var(tillamook,ddof=1)+(len(newport)-1)*np.var(newport,ddof=1)+(len(petersburg)-1)*np.var(petersburg,ddof=1)

In [33]:
SSB=len(tillamook)*(np.mean(tillamook)-tot_mean)**2+len(newport)*(np.mean(newport)-tot_mean)**2+len(petersburg)*(np.mean(petersburg)-tot_mean)**2

In [34]:
N=len(tillamook)+len(newport)+len(petersburg)

N

25

In [35]:
MSW=SSW/22

MSB=SSB/2

In [36]:
F=MSB/MSW

F

11.200832489917024

In [38]:
p=1-f.cdf(F,2,22)

p

0.0004418325189269323

In [39]:
scs.f_oneway(tillamook, newport, petersburg)

F_onewayResult(statistic=11.200832489917023, pvalue=0.00044183251892694495)

<h3>p-värde i butikslayoutexemplet</h3>

In [3]:
1-scs.f.cdf(8.1199,2,27)

0.0017337288783945137

<h3>Butikslauyout med scipy stats</h3>

In [9]:
#Customers, Major, Gig = None, None, None
Store_Layout, Interaction, Blockdata = None, None, None
with pd.ExcelFile(r"\\home.gu.gu.se\home-XS$\xsumat\Documents\ITHS\Data\Kap9.xlsx") as reader:
        Store_Layout=pd.read_excel(reader, sheet_name='Store_Layout')
        Interaction=pd.read_excel(reader, sheet_name='Interaction')
        Blockdata=pd.read_excel(reader, sheet_name='Blockdata')

In [5]:
Store_Layout.head()

,Layout 1,Layout 2,Layout 3
0,1.3,2.0,2.6
1,1.8,2.2,2.3
2,2.4,1.8,2.3
3,2.1,2.4,2.6
4,1.7,2.2,2.7


In [8]:
Store_Layout.describe()

,Layout 1,Layout 2,Layout 3
count,10.000000,10.000000,10.000000
mean,1.920000,2.080000,2.420000
std,0.311983,0.325918,0.193218
min,1.300000,1.400000,2.100000
25%,1.725000,1.850000,2.300000
50%,1.950000,2.200000,2.450000
75%,2.100000,2.300000,2.575000
max,2.400000,2.400000,2.700000


In [6]:
scs.f_oneway(Store_Layout['Layout 1'], Store_Layout['Layout 2'], Store_Layout['Layout 3'])

F_onewayResult(statistic=8.119926199261991, pvalue=0.0017337005157635715)

<h3> För Tukeys metod behöver vi ett paket från statsmodels </h3>

In [7]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [26]:
#Store_Layout till långformat
df=Store_Layout.stack().reset_index(name='Score')
df

,level_0,level_1,Score
0,0,Layout 1,1.3
1,0,Layout 2,2.0
2,0,Layout 3,2.6
3,1,Layout 1,1.8
4,1,Layout 2,2.2
5,1,Layout 3,2.3
6,2,Layout 1,2.4
7,2,Layout 2,1.8
8,2,Layout 3,2.3
9,3,Layout 1,2.1


In [27]:

tukey = pairwise_tukeyhsd(endog=df['Score'],
                          groups=df['level_1'],
                          alpha=0.05)

#display results
print(tukey)

  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1   group2  meandiff p-adj   lower  upper  reject
-------------------------------------------------------
Layout 1 Layout 2     0.16 0.4296 -0.1541 0.4741  False
Layout 1 Layout 3      0.5 0.0014  0.1859 0.8141   True
Layout 2 Layout 3     0.34 0.0319  0.0259 0.6541   True
-------------------------------------------------------


<h3>Lön vs. yrkesgrupp och utbildning</h3> 

In [35]:
Interaction.head()

,Utbildningsnivå,Yrkesgrupp,Lön
0,No High School,Educational,20
1,No High School,Educational,25
2,No High School,Educational,22
3,High School,Educational,30
4,High School,Educational,35


In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols



In [36]:
#perform two-way ANOVA
model = ols('Lön ~ C(Utbildningsnivå) + C(Yrkesgrupp) + C(Utbildningsnivå):C(Yrkesgrupp)', data=Interaction).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Utbildningsnivå),20523.888889,3.0,658.520499,3.582203e-23
C(Yrkesgrupp),916.222222,2.0,44.096257,9.182657e-09
C(Utbildningsnivå):C(Yrkesgrupp),318.444444,6.0,5.108734,1.659488e-03
Residual,249.333333,24.0,NaN,NaN


<h3>Blockdata</h3>

In [11]:
#perform two-way ANOVA
model = ols('Lön ~ C(Yrkesgrupp) + C(Utbildningsnivå)', data=Blockdata).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Yrkesgrupp),579.500000,2.0,6.442866,0.032067
C(Utbildningsnivå),7632.916667,3.0,56.575046,0.000086
Residual,269.833333,6.0,NaN,NaN
